<a href="https://colab.research.google.com/github/HussamCheema/DermCNN/blob/master/DermCNNPart2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import random
import tensorflow as tf
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.optimizers import SGD
from keras import regularizers
from keras.layers.normalization import BatchNormalization
from keras import optimizers
from keras import applications

In [2]:
!git clone https://github.com/HussamCheema/DermCNN.git

Cloning into 'DermCNN'...
remote: Enumerating objects: 11342, done.
remote: Counting objects: 100% (11342/11342), done.
remote: Compressing objects: 100% (11342/11342), done.
remote: Total 11342 (delta 2), reused 11335 (delta 0), pack-reused 0
Receiving objects: 100% (11342/11342), 438.26 MiB | 33.15 MiB/s, done.
Resolving deltas: 100% (2/2), done.
Checking out files: 100% (11359/11359), done.


# Data Pre-Processing

In [0]:
data = []
labels = []

In [0]:
batch_size = 32
epochs = 40

In [0]:
for folder in range(1,8):
  for image in os.listdir(os.getcwd()+'/DermCNN/'+str(folder)+'/'):
    try:
      x = cv2.imread(os.getcwd()+'/DermCNN/'+str(folder)+'/'+image)
      x = cv2.resize(x, (150,150))
      data.append(x)
      labels.append(folder)
    except: pass


In [6]:
data = np.array(data)
labels = np.array(labels)
print(len(data))
print(len(labels))
print(data.shape)
print(labels.shape)

11355
11355
(11355, 150, 150, 3)
(11355,)


In [7]:
data = data.astype('uint8')
labels = keras.utils.to_categorical(labels, 8)
print(labels.shape)

(11355, 8)


#Shuffle Data


In [0]:
def shuffle(a, b):
    rng_state = np.random.get_state()
    np.random.shuffle(a)
    np.random.set_state(rng_state)
    np.random.shuffle(b)

In [0]:
for i in range(10):
    shuffle(data,labels)

# Transfer Learning using Resnet model

In [12]:
model = keras.applications.resnet50.ResNet50(include_top=False, weights='imagenet', input_shape=(150, 150, 3))

/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


94658560/94653016 [==============================] - 3s 0us/step


In [0]:
# Freeze the layers which you don't want to train. Here I am freezing the first 5 layers.
for layer in model.layers[:5]:
    layer.trainable = False

In [20]:
#Adding custom Layers 
x = model.output
x = Flatten()(x)
x = Dropout(0.75)(x)
x = Dense(100,activation="relu")(x)
x = Dense(8,activation='softmax')(x)

# creating the final model 
model_final = Model(input = model.input, output = x)

model_final.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(lr=0.0005),
              metrics=['accuracy'])

X = model_final.fit(data,labels,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          shuffle=True,
          validation_split=0.2)

plt.plot(X.history['loss'])
plt.plot(X.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  


Train on 9084 samples, validate on 2271 samples
Epoch 1/40
9084/9084 [==============================] - 180s 20ms/step - loss: 1.0074 - acc: 0.7519 - val_loss: 2.7021 - val_acc: 0.2307
Epoch 2/40
9084/9084 [==============================] - 164s 18ms/step - loss: 1.0004 - acc: 0.7460 - val_loss: 2.1781 - val_acc: 0.3470
Epoch 3/40
9084/9084 [==============================] - 164s 18ms/step - loss: 0.5001 - acc: 0.8447 - val_loss: 1.9532 - val_acc: 0.5469
Epoch 4/40
9084/9084 [==============================] - 164s 18ms/step - loss: 0.4647 - acc: 0.8602 - val_loss: 7.0918 - val_acc: 0.1801
Epoch 5/40
9084/9084 [==============================] - 164s 18ms/step - loss: 0.3649 - acc: 0.8926 - val_loss: 6.0700 - val_acc: 0.1704
Epoch 6/40
8064/9084 [=========================>....] - ETA: 17s - loss: 0.3346 - acc: 0.8957

KeyboardInterrupt: ignored